In [14]:
import pandas as pd
import numpy as np
from collections import Counter

np.random.seed(13342)

In [15]:
A = np.array([1,2,3,4,5,6])
B = np.array([1,1,1,1,8,9])
C = np.array([2,2,3,4,5,5])
D = np.array([1,2,2,2,7,7]) 
E = np.array([1,2,2,3,4,9])
F = np.array([3,3,3,4,4,4])
G = np.array([2,2,2,2,4,9])
H = np.array([1,1,1,5,6,7])
I = np.array([2,2,3,4,4,6])


In [16]:
def compare_rolls(all_dice, names, num_iter = 1000):
    
    # Wins, ties, max roll, and num tie participant storage
    counts = Counter()
    tie_counts = Counter()
    max_roll_counts = Counter()
    xway_tie_counts = Counter()
    
    for i in range(num_iter):

        # Random choice
        rolls = list(map(np.random.choice, all_dice))
    
        # Store rolls, get max, and loc winning dice
        roll_storage = pd.DataFrame(data = rolls, index = names, columns = ["Roll"])
        max_roll = roll_storage.max().item()
        winners = roll_storage.loc[roll_storage['Roll'] == max_roll]
        
        # Count distribution of winning roll
        max_roll_counts[max_roll] += 1
        
        # Count distribution of ties with shared equity and wins
        if len(winners.index) > 1:
            xway_tie_counts[len(winners.index)] += 1
            for i in winners.index:
                tie_counts[i] += 1 / len(winners.index)
        else:
            for i in winners.index:
                counts[i] += 1
    
    return counts, tie_counts, max_roll_counts, xway_tie_counts

In [17]:
list_of_dice = [A,B,C,D,E,F,G,H,I]
list_of_names = ["A", "B", "C", "D", "E", "F", "G", "H", "I"]

In [18]:
wins, ties, max_rolls, xway_ties = compare_rolls(list_of_dice, list_of_names, num_iter = 10000)
total = wins + ties

sorted_wins = wins.most_common()
sorted_ties = ties.most_common()
sorted_total = total.most_common()
print("Wins only:")
display(sorted_wins)
print("\nShared ties included:")
display(sorted_total)

Wins only:


[('B', 2296),
 ('D', 1249),
 ('E', 1183),
 ('G', 1171),
 ('H', 1125),
 ('A', 497),
 ('C', 316),
 ('I', 309),
 ('F', 70)]


Shared ties included:


[('B', 2546.3333333333335),
 ('E', 1474.7),
 ('G', 1446.2333333333333),
 ('H', 1394.1666666666667),
 ('D', 1369.5),
 ('A', 683.1500000000002),
 ('C', 465.8166666666666),
 ('I', 447.69999999999993),
 ('F', 172.39999999999986)]

In [19]:
print("Max roll distribution:")
display(max_rolls.most_common())
print("\nXway tie distribution:")
display(xway_ties.most_common())

Max roll distribution:


[(9, 4214), (7, 2049), (8, 1146), (6, 1140), (5, 900), (4, 494), (3, 57)]


Xway tie distribution:


[(2, 1541), (3, 215), (4, 26), (5, 2)]

## Testing with Optimal Die

In [20]:
opt = np.array([1,1,1,1,8,9])
opt_list_of_dice = [A,B,C,D,E,F,G,H,I,opt]
opt_names = list_of_names + ["opt"]

In [21]:
o_wins, o_ties, o_max_rolls, o_xway_ties = compare_rolls(opt_list_of_dice, opt_names, num_iter = 5000)
o_total = o_wins + o_ties

o_sorted_wins = o_wins.most_common()
o_sorted_ties = o_ties.most_common()
o_sorted_total = o_total.most_common()
print("Wins only:")
display(o_sorted_wins)
print("\nShared ties included:")
display(o_sorted_total)

Wins only:


[('opt', 876),
 ('B', 869),
 ('G', 518),
 ('E', 489),
 ('D', 419),
 ('H', 335),
 ('A', 165),
 ('I', 103),
 ('C', 98),
 ('F', 20)]


Shared ties included:


[('opt', 1091.2500000000002),
 ('B', 1075.9166666666667),
 ('G', 685.4000000000001),
 ('E', 655.9500000000002),
 ('D', 463.5),
 ('H', 429.8333333333333),
 ('A', 229.95000000000002),
 ('I', 158.56666666666672),
 ('C', 151.98333333333335),
 ('F', 57.650000000000006)]

In [22]:
print("Max roll distribution:")
display(o_max_rolls.most_common())
print("\nXway tie distribution:")
display(o_xway_ties.most_common())

Max roll distribution:


[(9, 2610), (8, 865), (7, 669), (6, 372), (5, 289), (4, 171), (3, 24)]


Xway tie distribution:


[(2, 952), (3, 140), (4, 14), (5, 2)]

# Bonus Question

Let $\vec S$ be our vector of values for the dice

$$ \max_\vec S \Pr(\text{Win} | \vec S)$$

Consider all possible outcomes of our dice x.
$$ \Pr(\text{Win} | \vec S) = \sum_{x\in \vec S} \Pr(\text{Win} | \vec S, x) \Pr(x| \vec S) $$

$\Pr(x| \vec S) = \frac{1}{6}$ since each side is as equally likely. Also if given x we do not need S

$$ \Pr(\text{Win} | \vec S) = \frac{1}{6} \sum_{x\in \vec S} \Pr(\text{Win} | x) $$

Let y denote the maximum value of the other dice

$$\Pr(\text{Win} | x) = \Pr(y <= x) = \sum_{y=0}^x \Pr(y)$$

Finally we got back to our original goal
$$ \max_\vec S \frac{1}{6} \sum_{x\in \vec S} \sum_{y=0}^x \Pr(y) $$
$$ \max_\vec S \sum_{x\in \vec S} \sum_{y=0}^x \Pr(y) $$
We will use grid search to find the optimal value, and use the simulation results as an estimate for $\Pr(y)$

In [23]:
# Get an array for Pr(y)
max_rolls_distr = np.array(max_rolls.most_common(),dtype=int)
max_rolls_distr = np. append(max_rolls_distr, np.zeros((3,2),dtype=int), 0)
max_rolls_distr = max_rolls_distr[::-1]
max_rolls_distr = max_rolls_distr[:,1]
max_rolls_distr = max_rolls_distr / sum(max_rolls_distr)
max_rolls_distr

array([0.    , 0.    , 0.    , 0.0057, 0.0494, 0.09  , 0.114 , 0.1146,
       0.2049, 0.4214])

In [24]:
# Get all possible dice
import itertools

numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9]
target = 21
dice = [seq for i in range(len(numbers), 0, -1)
          for seq in itertools.combinations_with_replacement(numbers, i)
          if sum(seq) == target and len(seq) == 6]

dice = np.array(dice)
dice.shape

(81, 6)

In [25]:
max_val = 0
max_dice = np.zeros(6)

for s in dice:
    val = 0
    for x in s:
        for y in range(0,x+1):
            val += max_rolls_distr[y]
    if val > max_val:
        max_val = val
        max_dice = s
    elif val == max_val:
        max_dice = list(max_dice)
        max_dice.append(s)

        
print("Best Dice:")
print(max_dice)

print("Probability of winning:")
print(max_val/6)

Best Dice:
[1 1 1 1 8 9]
Probability of winning:
0.2631
